In [132]:
import pandas as pd
import numpy as np
import glob
import os

In [133]:
arquivo_csv_1 = ('data/processos_5.csv')
df_1 = pd.read_csv(arquivo_csv_1, sep='#', encoding='utf-8')
print(df_1.shape)

C:\Users\jcpsrodrigues\AppData\Local\Temp\ipykernel_3016\3268570998.py:2: DtypeWarning: Columns (5,20,43,54,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1 = pd.read_csv(arquivo_csv_1, sep='#', encoding='utf-8')


(2012271, 64)


In [134]:
pendentes = df_1[
    (df_1['serventia'] == '1º Juizado Especial Criminal') & 
    (df_1['comarca'] == 'APARECIDA DE GOIÂNIA')
]['data_baixa'].isna().sum()

print(f"Quantidade de valores nulos em 'data_baixa': {pendentes}")

Quantidade de valores nulos em 'data_baixa': 1327


In [135]:
baixados = df_1[
    (df_1['serventia'] == '1º Juizado Especial Criminal') & 
    (df_1['comarca'] == 'APARECIDA DE GOIÂNIA')
]['data_baixa'].notna().sum()

print(f"Quantidade de valores não nulos em 'data_baixa': {baixados}")

Quantidade de valores não nulos em 'data_baixa': 1513


In [136]:
# Limpeza e Tratamento
# Verificar o nome correto das colunas (pode haver diferenças de acentuação ou espaços)
colunas = df_1.columns.tolist()

# Encontrar as colunas de data corretamente
coluna_serventia = [col for col in colunas if 'serventia' in col.lower()][0]
coluna_distribuicao = [col for col in colunas if 'data_distribuicao' in col.lower()][0]
coluna_baixa = [col for col in colunas if 'data_baixa' in col.lower()][0]
coluna_area_acao = [col for col in colunas if 'nome_area_acao' in col.lower()][0]
coluna_processo_id = [col for col in colunas if 'processo_id' in col.lower()][0]
coluna_comarca = [col for col in colunas if 'comarca' in col.lower()][0]

# Renomear colunas para garantir consistência
df = df_1.rename(columns={
coluna_distribuicao: 'data_distribuicao',
coluna_baixa: 'data_baixa',
coluna_area_acao: 'nome_area_acao',
coluna_processo_id: 'processo_id',
coluna_comarca: 'comarca',
coluna_serventia: 'serventia'
})

In [137]:
# Gerando o Dataframe 1:
# 0) Garantir datetime
df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce')
df['data_baixa'] = pd.to_datetime(df['data_baixa'], errors='coerce')

keys = ['comarca', 'serventia'] # Altere aqui para fazer os agrupamentos

# 1) Cópia 
df_f = df.copy()

# 2.1) Referência anual
df_f['ano_dist'] = df_f['data_distribuicao'].dt.to_period('Y')
df_f['ano_baixa'] = df_f['data_baixa'].dt.to_period('Y')

# 3.1) Fluxos anuais
# Distribuídos por ano (fluxo)
dist_ano_base = (
    df_f.groupby(keys + ['ano_dist'])
        .size()
        .rename('Distribuidos_fluxo_anual')
        .reset_index()
        .rename(columns={'ano_dist': 'ano_ref'})
)

# Baixados por ano (fluxo)
baix_ano_base = (
    df_f.dropna(subset=['data_baixa'])
        .groupby(keys + ['ano_baixa'])
        .size()
        .rename('Baixados_fluxo_anual')
        .reset_index()
        .rename(columns={'ano_baixa': 'ano_ref'})
)

# Calcular valores acumulados
# Distribuídos acumulados
dist_acum = dist_ano_base.copy()
dist_acum = dist_acum.sort_values(['comarca', 'serventia', 'ano_ref'])
dist_acum['Distribuidos_acum_ano'] = dist_acum.groupby(keys)['Distribuidos_fluxo_anual'].cumsum()

# Baixados acumulados
baix_acum = baix_ano_base.copy()
baix_acum = baix_acum.sort_values(['comarca', 'serventia', 'ano_ref'])
baix_acum['Baixados_acum_ano'] = baix_acum.groupby(keys)['Baixados_fluxo_anual'].cumsum()

# Pendentes acumulados = Distribuídos acumulados - Baixados acumulados
pend_acum = dist_acum[keys + ['ano_ref', 'Distribuidos_acum_ano']].copy()
pend_acum = pend_acum.merge(
    baix_acum[keys + ['ano_ref', 'Baixados_acum_ano']], 
    on=keys + ['ano_ref'], 
    how='left'
)
pend_acum['Baixados_acum_ano'] = pend_acum['Baixados_acum_ano'].fillna(0).astype(int)
pend_acum['Pendentes_acum_ano'] = pend_acum['Distribuidos_acum_ano'] - pend_acum['Baixados_acum_ano']

# 6.1) Dataframe anual (APENAS COM AS COLUNAS SOLICITADAS)
df_global_anual = dist_ano_base[['comarca', 'serventia', 'ano_ref', 'Distribuidos_fluxo_anual']].copy()

# Adicionar Baixados_fluxo_anual
df_global_anual = df_global_anual.merge(
    baix_ano_base[['comarca', 'serventia', 'ano_ref', 'Baixados_fluxo_anual']],
    on=['comarca', 'serventia', 'ano_ref'],
    how='left'
)

# Adicionar Pendentes_acum_ano
df_global_anual = df_global_anual.merge(
    pend_acum[['comarca', 'serventia', 'ano_ref', 'Pendentes_acum_ano']],
    on=['comarca', 'serventia', 'ano_ref'],
    how='left'
)

# Adicionar Distribuidos_acum_ano (apenas para cálculo, depois removemos)
df_global_anual = df_global_anual.merge(
    dist_acum[['comarca', 'serventia', 'ano_ref', 'Distribuidos_acum_ano']],
    on=['comarca', 'serventia', 'ano_ref'],
    how='left'
)

# Preencher NaN com 0 e Converter para int
df_global_anual['Baixados_fluxo_anual'] = df_global_anual['Baixados_fluxo_anual'].fillna(0).astype(int)
df_global_anual['Pendentes_acum_ano'] = df_global_anual['Pendentes_acum_ano'].fillna(0).astype(int)
df_global_anual['Distribuidos_fluxo_anual'] = df_global_anual['Distribuidos_fluxo_anual'].astype(int)
df_global_anual['Distribuidos_acum_ano'] = df_global_anual['Distribuidos_acum_ano'].astype(int)

# Renomear colunas EXATAMENTE como solicitado
df_global_anual = df_global_anual.rename(columns={
    'Distribuidos_fluxo_anual': 'Distribuidos_ano',
    'Baixados_fluxo_anual': 'Baixados_ano',
    'Pendentes_acum_ano': 'Pendentes_ano'
})

# Calcular taxa anual CORRETA: Pendentes_ano / Distribuidos_acum_ano
df_global_anual['Taxa_Cong_anual (%)'] = np.where(
    df_global_anual['Distribuidos_acum_ano'] > 0,
    (df_global_anual['Pendentes_ano'] / df_global_anual['Distribuidos_acum_ano']) * 100,
    0
).round(2)

# Remover a coluna auxiliar Distribuidos_acum_ano
df_global_anual = df_global_anual.drop('Distribuidos_acum_ano', axis=1)

# Ordenar por comarca, serventia e ano
df_global_anual = df_global_anual.sort_values(['comarca', 'serventia', 'ano_ref'])

In [139]:
df_global_anual[(df_global_anual['ano_ref'].dt.year == 2024)].head()

,comarca,serventia,ano_ref,Distribuidos_ano,Baixados_ano,Pendentes_ano,Taxa_Cong_anual (%)
28,ABADIÂNIA,Vara Judicial,2024,1785,2335,2875,55.18
62,ALEXÂNIA,Vara Judicial,2024,4730,5162,6842,57.00
98,ALVORADA DO NORTE,Vara Judicial,2024,2044,2539,4505,63.96
126,ANÁPOLIS,1ª Vara Criminal,2024,1003,2151,2527,54.02
163,ANÁPOLIS,1ª Vara Cível,2024,1759,2001,4266,68.07


In [140]:
df_global_anual[
    (df_global_anual['comarca'] == 'APARECIDA DE GOIÂNIA') & 
    (df_global_anual['serventia'] == '1º Juizado Especial Criminal') #&
    #(df_global_anual['ano_ref'].dt.year == 2025)
].head(15)

,comarca,serventia,ano_ref,Distribuidos_ano,Baixados_ano,Pendentes_ano,Taxa_Cong_anual (%)
755,APARECIDA DE GOIÂNIA,1º Juizado Especial Criminal,2011,21,0,21,100.00
756,APARECIDA DE GOIÂNIA,1º Juizado Especial Criminal,2012,33,0,54,100.00
757,APARECIDA DE GOIÂNIA,1º Juizado Especial Criminal,2013,45,0,99,100.00
758,APARECIDA DE GOIÂNIA,1º Juizado Especial Criminal,2014,51,0,150,100.00
759,APARECIDA DE GOIÂNIA,1º Juizado Especial Criminal,2015,37,0,187,100.00
760,APARECIDA DE GOIÂNIA,1º Juizado Especial Criminal,2016,27,0,214,100.00
761,APARECIDA DE GOIÂNIA,1º Juizado Especial Criminal,2017,34,0,248,100.00
762,APARECIDA DE GOIÂNIA,1º Juizado Especial Criminal,2018,26,0,274,100.00
763,APARECIDA DE GOIÂNIA,1º Juizado Especial Criminal,2019,51,0,325,100.00
764,APARECIDA DE GOIÂNIA,1º Juizado Especial Criminal,2020,109,0,434,100.00


In [141]:
df_2024 = df_global_anual[df_global_anual['ano_ref'].astype(str) == '2024']
df_2024.head(10)

,comarca,serventia,ano_ref,Distribuidos_ano,Baixados_ano,Pendentes_ano,Taxa_Cong_anual (%)
28,ABADIÂNIA,Vara Judicial,2024,1785,2335,2875,55.18
62,ALEXÂNIA,Vara Judicial,2024,4730,5162,6842,57.00
98,ALVORADA DO NORTE,Vara Judicial,2024,2044,2539,4505,63.96
126,ANÁPOLIS,1ª Vara Criminal,2024,1003,2151,2527,54.02
163,ANÁPOLIS,1ª Vara Cível,2024,1759,2001,4266,68.07
165,ANÁPOLIS,1º CENTRO JUDICIÁRIO DE SOLUÇÃO DE CONFLITOS E...,2024,238,2,253,99.22
176,ANÁPOLIS,1º Juizado Especial Criminal,2024,1121,1334,1740,56.60
192,ANÁPOLIS,1º Juizado Especial Cível,2024,3963,5183,2534,32.84
222,ANÁPOLIS,2ª Vara Criminal,2024,1021,1729,1891,52.24
261,ANÁPOLIS,2ª Vara Cível,2024,1775,2262,3910,63.35


In [142]:
len(df_2024)

368

# Gerador de Arquivo

In [130]:
df_2025 = df_global_anual[df_global_anual['ano_ref'].astype(str) == '2025']
os.makedirs("results", exist_ok=True)
df_2025.to_csv("results/tx_2025.csv", sep=",", index=False, encoding="utf-8-sig")
